### Loading libraries

In [22]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
import datetime
import random
from tqdm.notebook import tqdm
from PIL import Image
from PIL import ImageEnhance
pd.set_option('display.max_colwidth', None)

### Segmentation

Defining Paths

In [23]:
def get_img(path_str):
    # reads at RGB
    return np.asarray(Image.open(path_str))

def show_img(img):
    plt.imshow(img)
    plt.show()

def show_img_gray(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))
    plt.show()

def get_gray(img):
    return cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

def apply_threshold(img):
    return cv2.threshold(img, np.mean(img), 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

def get_edge(img):
    return cv2.dilate(cv2.Canny(img, 0, 255), None)

def get_mask(edge_img):
    cnt = sorted(cv2.findContours(edge_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2], key=cv2.contourArea)[-1]
    mask_shape = edge_img.shape[0],edge_img.shape[1]
    mask = np.zeros(mask_shape, np.uint8)
    return cv2.drawContours(mask, [cnt],-1, 255, -1)




In [24]:
def edge_based_segmentation(img, show_image = True):
    edge_img = get_edge(apply_threshold(get_gray(img)))
    mask_img = get_mask(edge_img)
    if show_image : 
        show_img(img)
        show_img_gray(mask_img)
    return mask_img

### Perfomance Metric

In [25]:
# todo make it read image as binary
def read_true_mask(img_path_str):
    return cv2.cvtColor(cv2.imread(img_path_str),cv2.COLOR_RGB2GRAY)
    
def find_metrics(seg_mask_img, org_mask_img):
    f1, iou, pixacc = 0,0,0 
    intersection = np.sum( np.logical_and(seg_mask_img, org_mask_img) )
    union = np.sum ( np.logical_or(seg_mask_img, org_mask_img) )
    cnt_seg = np.count_nonzero(seg_mask_img)
    cnt_org = np.count_nonzero(org_mask_img)
    h,w = seg_mask_img.shape
    cnt_tot = h*w 
    cnt_true = intersection
    cnt_false = cnt_tot - union 

    f1 = 2 * intersection / (cnt_seg + cnt_org) 
    iou = intersection / union
    pixacc = (cnt_true + cnt_false) / cnt_tot

    return f1, iou, pixacc

def print_all(seg_mask_img, org_mask_img):
    f1, iou, pixacc = find_metrics(seg_mask_img, org_mask_img)
    print("f1 = ", f1, " iou = ", iou, " pixacc = ", pixacc)


### Applying above functions

In [26]:
img_path = './final_dataset/images'
mask_path = './final_dataset/masks'
out_path = './final_dataset/masks_edge/'

imgs = glob.glob(os.path.join(img_path, "*.png"))
masks = glob.glob(os.path.join(mask_path, "*.png")) 

imgs = sorted(imgs)
masks = sorted(masks)

num_images = len(masks)

# generate masks and save to file

In [27]:

f1_scores, iou_scores, pixacc_scores = [], [], []


for i in range(0, num_images):
    img = get_img(imgs[i])
    seg_mask_img = edge_based_segmentation(img, show_image=False)
    cv2.imwrite(out_path + 'img_' + str(i).zfill(4) + '.png',  seg_mask_img)

    org_mask_img = read_true_mask(masks[i])
    f1, iou, pixacc = find_metrics(seg_mask_img, org_mask_img)
    f1_scores.append(f1)
    iou_scores.append(iou)
    pixacc_scores.append(pixacc)

    if (i+1)%100 == 0 : 
        print("Processed image", str(i+1))


Processed image 100
Processed image 200
Processed image 300
Processed image 400
Processed image 500
Processed image 600
Processed image 700
Processed image 800
Processed image 900
Processed image 1000


In [28]:
sum_f1, sum_iou, sum_pixacc = 0,0,0 

for i in range(0, num_images):
    sum_f1 += f1_scores[i]
    sum_iou += iou_scores[i]
    sum_pixacc += pixacc_scores[i]

print("Average F1 score = ", str(sum_f1 / num_images))
print("Average IOU score = ", str(sum_iou / num_images))
print("Average Pixacc score = ", str(sum_pixacc / num_images))

Average F1 score =  0.354179420232433
Average IOU score =  0.27440526972853246
Average Pixacc score =  0.8489825886625242
